### Chapter 5. The life-cycle model and intertemporal choice

Reference: 
Fehr, H. and Kindermann, F. 2018. The overlapping generations model. <i>Introduction to Computational Economics Using Fortran.</i> Oxford University Press. pp205-224.

In [1]:
##### Program 5.1 Optimal savings in a certain world

# Import package
import numpy as np
from numpy.linalg import inv
from scipy.optimize import minimize

# Parameters
w = 1.0         #labour income
R = 1.0         #return on savings, R=1+r
β = 1.0         #discount factor
γ = 0.5         #relative risk aversion, when γ=0, the utility function is CARA. 
egam = 1 - 1/γ  #denominator of utility function, power to consumption

# Endogenous variables
a = np.zeros(3) # Savings a1, a2 and a3 are 3-dimensional arrays
c = np.zeros(3) # Consumption c1, c2 and c3 are 3-dimensional arrays

### Define functions
def utility(x): # Input x is a 2-dimentional array (our chocie variables are a2 and a3 (a1=0))
    """
    Program 5.1 function
    This function defines utilily of agent live for 3 periods
    input: assets for the second and third period, a_2 and a_3
    output: utility
    """
    global c    # store local c as global
    ## savings
    a[1:3] = x  # Set a[1] and a[2] be the inputs x[0] and x[1]
    
    ## consumption (insure consumption > 0)
    c[0] = w - a[1]              # First period c1=w-a2
    c[1] = R * a[1] + w - a[2]   # Second period c2=w+R*a2-a3
    c[2] = R * a[2]              # Third period c3=R*a3
    # Guarantees c>0
    c    = np.maximum(c,np.zeros(3)+1e-10) # Gurantee that consumption is positive
    ## calculate utility
    value = -(c[0] ** egam + β * c[1] ** egam + β ** 2 * c[2] ** egam)/egam ## Minimize the negative of utility
    return value

# Set the constraints for savings
low = np.zeros(2)
# Set the lower bound for savings a2 and a3 (greater than zero)
up = np.array([w, R * w + w]) 
# Set the upper bound for a2 and a3 (not exceed the maximum avaiable in periods 2 and periods 3)
bns = [(low[0],up[0]),(low[1],up[1])]

# Initial Guess
x0 = np.array([.1,.1])

# Minimize routine
res = minimize(utility, x0, bounds=bns)
a[1:3] = res.x             #result that minimizes utility function
c[0] = w - a[1]            #c1=w-a2 substituting a2 result
c[1] = R * a[1] + w - a[2] #c2=R*a2+w-a3 substituting a2, a3 result
c[2] = R * a[2]            #c3=R*a3 substituting a3 result

# Output #"%.2f" defines output format as float with 2 decimal points, %:string modulo operator
print('AGE   CONS   WAGE   INC    SAV')
print(1, '   ', "%.2f"%c[0], ' ', "%.2f"%w, ' ', "%.2f"%w, ' ', "%.2f"%a[1])
print(2, '   ', "%.2f"%c[1], ' ', "%.2f"%w, ' ', "%.2f"%(w+R*a[1]), ' ', "%.2f"%a[2])
print(3, '   ', "%.2f"%c[2], ' ', "%.2f"%0, ' ', "%.2f"%(R*a[2]), ' ', "%.2f"%0)

AGE   CONS   WAGE   INC    SAV
1     0.67   1.00   1.00   0.33
2     0.67   1.00   1.33   0.67
3     0.67   0.00   0.67   0.00


In [4]:
### Program: normal discretize and log-normal discretize
import numpy as np

def normal_discrete(n, mu = 0.0, sigma = 1.0):
    """
    This function returns a discretized normal distribution N(mu,sigma) with n nodes and n corresponding weights.
    Input:  n : number of nodes
            mu: mean of required normal distribution
            sigma: variance of required normal distribution
    Output: x: value of each node
            prob: weight corresponding to each node
    """
    mu_c = mu            #mean of the distribution
    sigma_c = sigma**0.5 #standard deviation of the distribution
    if sigma_c < 0.0:
        return print('error: sigma has negative value')
    pim4 = 1.0/(np.pi**0.25) #square root of standard deviation parameter ?
    m = int((n+1)/2)     #Since normal distribution is symmetric, find out how many nodes one side included.
    
    #Initialize output variables
    x = np.zeros(n)      #initialized nodes' value
    prob = np.zeros(n)   #initialized nodes' weight
    
    its = 0              #initial iteration number
    z1 = 0.0             #middle value storing computed z
    
    for i in range(m):   #numerical approximation of normal distribution, ref: Fehr & Kindermann (2018) toolbox
        if i == 0:
            z = (float(2*n+1))**0.5-1.85575*(float(2*n+1))**(-1.0/6.0)
        elif i == 1:
            z = z - 1.14*(float(n)**0.426)/z
        elif i == 2:
            z = 1.86*z + 0.86*x[0]
        elif i == 3:
            z= 1.91*z+0.91*x[1]
        else:
            z = 2.0*z+x[i-2]
        
        while its < 200:
            its = its + 1
            p1 = pim4
            p2 = 0.0
            for j in range(1,n+1): # for j = 1, ..., n
                p3 = p2
                p2 = p1
                p1 = z*(2.0/float(j))**0.5*p2-p3*(float(j-1)/float(j))**0.5
            pp = p2*(2.0*float(n))**0.5
            z1 = z
            z = z1-p1/pp
            if abs(z-z1)<1e-14:
                break
        if its>200:
            print('error: Coule not discretize normal distribution')
        x[n-1-i] = z
        x[i] = -z
        prob[i]= 2.0/pp**2
        prob[n-1-i] = prob[i]
         
    prob = prob/np.pi**0.5 #normalization
    x = x*2.0**0.5*sigma_c + mu_c
     
    return x, prob

def log_normal_discrete(n, mu = np.exp(0.5), sigma = np.exp(1.0)*(np.exp(1.0)-1.0)):
    """
    This function returns a discretized lognormal distribution logN(mu,sigma) with n nodes and n corresponding weights.
    Input:  n : number of nodes
            mu: mean of required lognormal distribution
            sigma: variance of required lognormal distribution
    Output: x: value of each node
            prob: weight corresponding to each node
    """
    mu_c = mu          #mean of distribution
    sigma_c = sigma    #standard deviation of distribution
    
    if sigma_c < 0.0:
        print('error: sigma has negative value')
    if mu_c <= 0.0:
        print('error: mu has zero or negative value')
    
    #Transfer from lognormal distribution to corresponding normal distribution
    sigma_c = np.log(1.0+sigma_c/mu_c**2) #mean of transfered normal distribution
    mu_c = np.log(mu_c)-0.5*sigma_c       #standard deviation of transfered normal distribution
    
    x = np.array(normal_discrete(n,mu_c,sigma_c))[0:1].reshape((n,)) #reshaping first column result to row
    x = np.exp(x) #transfer normal distributon discretized nodes to lognormal distribution values
    prob = np.array(normal_discrete(n,mu_c,sigma_c))[1:].reshape((n,)) #reshaping second column result to row
    return x, prob

In [3]:
#### Program 5.2 Utility function with wage risk
# Import package
import numpy as np
from scipy.optimize import minimize


# Parameters
μw = 1.0            #mean of labor income distribution
σw = 1.0            #variance of labor income distribution
n_w = 5             #number of discretized nodes
R = 1.0             #return on savings, R=1+r
β = 1.0             #discount factor
γ = 0.5             #relative risk aversion
egam = 1 - 1/γ      #denominator of utility function, power to consumption

# Initialize Endogenous Variables
a2 = .0             #asset holdinds at period 2
a3 = np.zeros(n_w)  #possible asset holdings at period 3
c1 = .0             #current consumption

### Discretize w:
wi = np.array(log_normal_discrete(n_w,μw,σw))[0:1].reshape((n_w,)) #discretized nodes' value
ω = np.array(log_normal_discrete(n_w,μw,σw))[1:].reshape((n_w,))   #discretized nodes' weight

# Defining utility function
def utility(x):
    """
    This is a 3 period utility function. 
    1st period: μw has been realized, 
                a2 choice variable as how much to save to period 2, 
                c1 consumption in period 1 is pinned down by choosing a2,
                no discount
    2nd period: discounted utility with unrealized c2, w, so all these are arrays representing possible values of each
    3rd period: discounted for 2 periods utility with unrealized c3, a3
    Input: asset holdings a2=x[0] and a3=x[1:]
    Output: negative expected value of the 3 period utility function in period 1
    """
    a2 = x[0]             #period 2 asset holding
    a3 = x[1:]            #period 3 asset holdings
    c1 = μw - a2          #c1=μw-a2 first period consumption
    c2 = R * a2 + wi - a3 #c2=R*a2+wi-a3 array of second period consumption with possible labor income values
    c3 = R * a3           #c3=R*a3 array of third period consumption
    # Guarantees c>0
    for i in range(n_w):
        c1    = max(c1   , 1e-10)
        c2[i] = max(c2[i], 1e-10)
        c3[i] = max(c3[i], 1e-10)
    # Calculate the expected utility of period 2 and 3
    expect = 0.0          #initialize expected utility
    for i in range(n_w):
        expect = expect + ω[i]*(c2[i]**egam+β*c3[i]**egam)/egam #summaries utility for each possible labor income values
    # Calculate the expected utility at period 1
    value = -(c1**egam/egam+β*expect) #negative of utility to minimize
    return value


# Bounds
low = np.zeros(1+n_w)  #initialize lower bound for a2 and a3
eps = 1.0e-8           #very small number
for i in range(1+n_w):
    low[i]=eps         #guarantees the lower bound is positive
up = np.zeros(1+n_w)   #initialize upper bound for a2 and 3a
up[0] = μw             #c1=μw-a2>0, so a2=μw-c1<μw. A positive c1 guarantees a2 has upper bound of μw.
up[-n_w:] = R*μw + wi  #c2=R*μw+wi-a3>0, so a3=R*μw+wi-c2<μw+wi.
bnds = [(low[i],up[i]) for i in range(1+n_w)] #summarize bounds for each a2 and possible a3.


# Maxmize the utility function
# Initial guess
x0 = np.zeros(1+n_w)
# Maximum
res = minimize(utility,x0,bounds=bnds) #minimize the negative of utility function with initial guess of a2, a3 given bounds


# Calculate c1
a2 = res.x[0]           #output the result of minimized negative utility (maximized utility), a2=x[0]
c1 = μw - a2            #c1=μw-a2, first period consumption, μw is realized, a2 is leftover from period 1 consumption
a3 = res.x[-n_w:]       #output the result of minimized negative utility (maximized utility), a3=x[1:]

# Calculate the expectation
def E(x):
    E = 0.0
    for i in range(len(x)):
        E = E + ω[i]*x[i]
    return E

# Calculate the standard deviation
def Std(x):
    std = 0.0
    for i in range(len(x)):
        std = std + ω[i]*x[i]**2
    std = (max(std-E(x)**2,0.0))**0.5
    return std


# Calculate other endogenous variables
c2 = R * a2 + wi - a3   #c2=R*a2+wi-a3 substituting a2, wi, a3 result
c3 = R * a3             #c3=R*a3 substituting a3 result


# Results
#"%.2f" defines output format as float with 2 decimal points, %:string modulo operator
print('-------------------')
print('First period:')
print('c1    = ', "%.2f"%c1   , 'a2      = ',"%.2f"%a2)
print('-------------------')
print('Second period:')
print('E(c2) = ', "%.2f"%E(c2), 'Std[c2] = ',"%.2f"%Std(c2))
print('-------------------')
print('Third period:')
print('E(c3) = ', "%.2f"%E(c3), 'Std[a3] = ',"%.2f"%Std(a3))
print('-------------------')
print('AGE   CONS   WAGE   INC    SAV')
print(1, '   ', "%.2f"%c1, ' ', "%.2f"%μw, ' ', "%.2f"%μw, ' ', "%.2f"%a2,'(MEAN)')
print(' ','   ',"%.2f"%0, ' ', "%.2f"%0, ' ', "%.2f"%0, ' ', "%.2f"%0,'(STD)')
print(2, '   ', "%.2f"%E(c2), ' ', "%.2f"%E(wi), ' ', "%.2f"%E(wi+R*a2), ' ', "%.2f"%E(a3),'(MEAN)')
print(' ','   ',"%.2f"%Std(c2), ' ', "%.2f"%Std(wi), ' ', "%.2f"%Std(wi+R*a2), ' ', "%.2f"%Std(a3),'(STD)')
print(3, '   ', "%.2f"%E(c3), ' ', "%.2f"%0, ' ', "%.2f"%E(R*a3), ' ', "%.2f"%0,'(MEAN)')
print(' ','   ',"%.2f"%Std(c3), ' ', "%.2f"%0, ' ', "%.2f"%Std(R*a3), ' ', "%.2f"%0,'(STD)')

-------------------
First period:
c1    =  0.53 a2      =  0.47
-------------------
Second period:
E(c2) =  0.74 Std[c2] =  0.50
-------------------
Third period:
E(c3) =  0.74 Std[a3] =  0.50
-------------------
AGE   CONS   WAGE   INC    SAV
1     0.53   1.00   1.00   0.47 (MEAN)
      0.00   0.00   0.00   0.00 (STD)
2     0.74   1.00   1.47   0.74 (MEAN)
      0.50   1.00   1.00   0.50 (STD)
3     0.74   0.00   0.74   0.00 (MEAN)
      0.50   0.00   0.50   0.00 (STD)


In [4]:
#### Program 5.3 optimal savings with wage risk and risky capital returns
# Import package
import numpy as np
from scipy.optimize import minimize

# Parameters
μw = 1.0            #mean of labor income distribution
μR = 1.0            #mean of capital return distribution
σw = 0.4            #variance of labor income distribution
σR = 0.4            #variance of capital return distribution
n_w = 5             #number of discretized nodes for labor income
n_R = 5             #number of discretized nodes for capital return
β = 1.0             #discount factor
γ = 0.5             #relative risk aversion
egam = 1 - 1/γ      #denominator of utility function, power to consumption

# Initialize Endogenous Variables
a2 = .0                  #asset holdinds at period 2
a3 = np.zeros((n_w,n_R)) #possible asset holdings at period 3
c1 = .0                  #current consumption

### Discretize w and R:
wi = np.array(log_normal_discrete(n_w,μw,σw))[0:1].reshape((n_w,))  #discretized nodes' value for w
ωw = np.array(log_normal_discrete(n_w,μw,σw))[1:].reshape((n_w,))   #discretized nodes' weight for w
Ri = np.array(log_normal_discrete(n_R,μR,σR))[0:1].reshape((n_R,))  #discretized nodes' value for R
ωR = np.array(log_normal_discrete(n_R,μR,σR))[1:].reshape((n_R,))   #discretized nodes' weight for R

# Defining utility function
def utility(x):
    """
    This is a 3 period utility function. 
    1st period: μw,μR have been realized, 
                a2 choice variable as how much to save to period 2, 
                c1 consumption in period 1 is pinned down by choosing a2,
                no discount
    2nd period: discounted utility with unrealized c2, w, so all these are arrays representing possible values of each
    3rd period: discounted for 2 periods utility with unrealized c3, a3
    Input: asset holdings a2=x[0] and a3=x[1:]
    Output: negative expected value of the 3 period utility function in period 1
    """
    a2 = x[0]                     #period 2 asset holding
    a3 = x[1:].reshape((n_w,n_R)) #period 3 asset holdings, reshape to a n_w by n_R matrix
    c1 = μw - a2                  #c1=μw-a2 first period consumption
    # Calculate the expected utility of period 2 and 3
    expect = 0.0                  #initialize expected utility
    for i in range(n_w):          #loop through each labor income node in period 2
        for j in range(n_R):      #loop through each capital return node in period 2
            for k in range(n_R):  #loop through each capital return node in period 3
                #expectation over all possible labor income in period 2 and possible capital return in period 2 and 3
                expect = expect + ωw[i]*ωR[j]*ωR[k]*((Ri[j]*a2+wi[i]-a3[i,j])**egam+\
                                                     β*(Ri[k]*a3[i,j])**egam)/egam 
    value = -(c1**egam/egam+β*expect) #negative of utility to minimize
    return value


# Bounds
low = np.zeros(1+n_w*n_R)          #initialize lower bound for a2 and a3
eps = 1.0e-14                      #very small number
for i in range(1+n_w*n_R):
    low[i]=eps                     #guarantees the lower bound is positive
up = np.zeros(1+n_w*n_R)           #initialize upper bound for a2 and a3
up[0] = μw                         #upper bound for a2 is determined by c1>0, a2=μw-c1<0
k = 1                              #initialize iteration round, first set of wi and Ri
for i in range(n_w):               #loop through labor income nodes
    for j in range(n_R):           #loop through capital return nodes
        up[k] = Ri[j]*max(wi)+wi[i]#c2=Ri*μw+wi-a3>0, so a3=Ri*μw+wi-c2<Ri*μw+wi. 
        k = k+1                    #Iteration up, next iteration (k+1) set of wi and Ri combination
bnds = [(low[i],up[i]) for i in range(1+n_w*n_R)] #assign upper bounds for a2 and each bound set for a3


# Maxmize the utility function
# Initial guess
x0 = np.zeros(1+n_w*n_R)
# Maximum
res = minimize(utility,x0,bounds=bnds) #minimize the negative of utility function with initial guess of a2, a3 given bounds

# Calculate c1
a2 = res.x[0]       #output the result of minimized negative utility (maximized utility),a2=x[0]
c1 = w - a2         #c1=μw-a2, first period consumption, μw is realized, a2 is leftover from period 1 consumption
a3 = res.x[1:].reshape((n_w,n_R)) #output the result of minimized negative utility (maximized utility), a3=x[1:] 
                                  #and reshaping into n_w by n_R matrix


# Calculate the expectation
# Expectation of 2D array
def E2(x):
    E = 0.0
    for i in range(n_w):
        for j in range(n_R):
            for k in range(n_R):
                E = E + ωw[i]*ωR[j]*ωR[k]*x[i,j]
    return E

# Expectation of 3D array
def E3(x):
    E = 0.0
    for i in range(n_w):
        for j in range(n_R):
            for k in range(n_R):
                E = E + ωw[i]*ωR[j]*ωR[k]*x[i,j,k]
    return E

# Calculate the standard deviation
# Standard deviation of 2D array
def Std2(x):
    std = 0.0
    for i in range(n_w):
        for j in range(n_R):
            for k in range(n_R):
                std = std + ωw[i]*ωR[j]*ωR[k]*x[i,j]**2
    std = max(std-E2(x)**2.0,0.0)**0.5
    return std

# Standard deviation of 3D array
def Std3(x):
    std = 0.0
    for i in range(n_w):
        for j in range(n_R):
            for k in range(n_R):
                std = std + ωw[i]*ωR[j]*ωR[k]*x[i,j,k]**2
    std = max(std-E3(x)**2.0,0.0)**0.5
    return std

# Calculate other endogenous variables
c2 = np.zeros((n_w,n_R)) #Initialize
for i in range(n_w):     #loop for possible labor income nodes in period 2
    for j in range(n_R): #loop for possible capital return nodes in period 2
        c2[i,j] = Ri[j]*a2 + wi[i] -a3[i,j]

c3 = np.zeros((n_w,n_R,n_R)) #Initialize
for i in range(n_w):         #loop for possible labor income nodes in period 2
    for j in range(n_R):     #loop for possible capital return nodes in period 2
        for k in range(n_R): #loop for possible capital return nodes in period 3
            c3[i,j,k] = Ri[k]*a3[i,j]

# Calculate inc in the second period
inc2 = np.zeros((n_w,n_R)) #Initialize
for i in range(n_w):     #loop for possible labor income nodes in period 2
    for j in range(n_R): #loop for possible capital return nodes in period 2
        inc2[i,j] = Ri[j]*a2 + wi[i]
        
# Calculate inc in the second period
inc3 = np.zeros((n_w,n_R,n_R)) #Initialize
for i in range(n_w):         #loop for possible labor income nodes in period 2
    for j in range(n_R):     #loop for possible capital return nodes in period 2
        for k in range(n_R): #loop for possible capital return nodes in period 3
            inc3[i,j,k] = Ri[k]*a3[i,j]
# Results
print('-------------------')
print('First period:')
print('c1    = ',"%.2f"%c1    , 'a2      = ',"%.2f"%a2)
print('-------------------')
print('Second period:')
print('E(c2) = ',"%.2f"%E2(c2), 'Std[c2] = ',"%.2f"%Std2(c2), \
      '\nE(a3) = ',"%.2f"%E2(a3), 'Std[a3] = ',"%.2f"%Std2(a3))
print('-------------------')
print('Third period:')
print('E(c3) = ',"%.2f"%E3(c3), 'Std[c3] = ',"%.2f"%Std3(c3))
print('-------------------')
print('AGE   CONS   WAGE   INC    SAV')
print(1, '   ', "%.2f"%c1, ' ', "%.2f"%μw, ' ', "%.2f"%μw, ' ', "%.2f"%a2,'(MEAN)')
print(' ','   ',"%.2f"%0, ' ', "%.2f"%0, ' ', "%.2f"%0, ' ', "%.2f"%0,'(STD)')
print(2, '   ', "%.2f"%E2(c2), ' ', "%.2f"%E(wi), ' ', "%.2f"%E2(inc2), ' ', "%.2f"%E2(a3),'(MEAN)')
print(' ','   ',"%.2f"%Std2(c2), ' ', "%.2f"%Std(wi), ' ', "%.2f"%Std2(inc2), ' ', "%.2f"%Std2(a3),\
      '(STD)')
print(3, '   ', "%.2f"%E3(c3), ' ', "%.2f"%0, ' ', "%.2f"%E3(inc3), ' ', "%.2f"%0,'(MEAN)')
print(' ','   ',"%.2f"%Std3(c3), ' ', "%.2f"%0, ' ', "%.2f"%Std3(inc3), ' ', "%.2f"%0,'(STD)')
print('-------------------')
print('E(w)=',"%.2f"%E(wi),'  ','Var(w)=',"%.2f"%Std(wi)**2)
print('E(R)=',"%.2f"%E(Ri),'  ','Var(R)=',"%.2f"%Std(Ri)**2)

-------------------
First period:
c1    =  0.56 a2      =  0.44
-------------------
Second period:
E(c2) =  0.66 Std[c2] =  0.32 
E(a3) =  0.78 Std[a3] =  0.37
-------------------
Third period:
E(c3) =  0.78 Std[c3] =  0.66
-------------------
AGE   CONS   WAGE   INC    SAV
1     0.56   1.00   1.00   0.44 (MEAN)
      0.00   0.00   0.00   0.00 (STD)
2     0.66   1.00   1.44   0.78 (MEAN)
      0.32   0.63   0.69   0.37 (STD)
3     0.78   0.00   0.78   0.00 (MEAN)
      0.66   0.00   0.66   0.00 (STD)
-------------------
E(w)= 1.00    Var(w)= 0.40
E(R)= 1.00    Var(R)= 0.40


In [2]:
### Program: 2-dimensional log-normal discretize
from scipy.linalg import cholesky

def log_normal_discrete_2(n, mu = np.exp(0.5*np.ones(2)), \
                          sigma = np.exp(1.0)*(np.exp(np.ones(2))-1.0), rho = 0.0):
    """
    This function returns a discretized 2-dimensional lognormal distribution logN(mu,sigma) 
            with n nodes and n corresponding weights.
    Input:  n : a 2D array of numbers nodes for each distribution
            mu: a 2D array of means of lognormal distribution
            sigma: a 2D array of variance of lognormal distribution
            rho: covariance of 2D lognormal distribution
    Output: x: value of each node
            prob: weight corresponding to each node
    """
    # Initialize parameters
    mu_c = mu      #means of the distribution
    rho_c = rho    #correlation of distribution
    sig_c = sigma  #variances of distribution
    
    if np.amin(mu_c) <= 0.0:
        print('error: mu has zero or negative value')
    if np.amin(sig_c) < 0.0:
        print('error: sigma has negative value')
    if abs(rho) > 1.0:
        print('error: rho is outside -1 to 1')
    
    # Get expectation and variance
    sig_c = np.log(1.0+sig_c/(mu_c**2.0))  #transfer lognormal variances to normal distribution variances
    mu_c = np.log(mu_c)-0.5*sig_c          #transfer lognormal means to normal distribution means
    
    # Set up covariance matrix
    sigma_c = np.zeros((2,2))       #Initialize covariance matrix
    sigma_c[0,0] = sig_c[0]         #var(1,1)
    sigma_c[1,1] = sig_c[1]         #var(2,2)
    sigma_c[0,1] = np.log(rho_c*(np.exp(sig_c[0])-1.0)**0.5*(np.exp(sig_c[1])-1.0)**0.5+1.0) #cov(1,2)
    sigma_c[1,0] = sigma_c[0,1]     #cov(2,1)=cov(1,2)
    
    # Check whther sigma is symmetric
    if np.amin(np.absolute(sigma_c.T-sigma_c))>1e-20:
        print('error: Variance-Covariance matrix is not symmetric')
    
    # Get standard normal distributed random variables
    x1, p1 = normal_discrete(n[0],0.0,1.0)  #discretized nodes' value for row distribution
    x2, p2 = normal_discrete(n[1],0.0,1.0)  #discretized nodes' value for column distribution
    
    # Get joint distribution
    m = 0  #Initial value
    prob = np.zeros(n[0]*n[1])  #initialize weights on n[0] by n[1] nodes
    x = np.zeros((n[0]*n[1],2)) #initialize values on n[0] by n[1] nodes
    for j in range(n[0]):       #loop for row distribution
        for k in range(n[1]):   #loop for column distribution
            prob[m] = p1[j]*p2[k] #joint probability
            x[m,0] = x1[j]      #nodes for row distribution
            x[m,1] = x2[k]      #nodes for column distribution
            m = m+1
            
    # Decompose car-cov matrix
    if np.amin(np.absolute(sig_c)) != 0.0:
        l = cholesky(sigma_c,lower = True)
    else:
        l = np.zeros((2,2))
        l[0,0] = sig_c[0]**0.5
        l[1,1] = sig_c[1]**0.5
    
    # Calcualte distribution
    x = x@(l.T)               #matrix multiplication of x and transposed lower triangular matrix
    x[:,0] = x[:,0] + mu_c[0] #scaling standard normal nodes with row distribution mean mu_c[0]
    x[:,1] = x[:,1] + mu_c[1] #scaling standard normal nodes with column distribution mean mu_c[1]
    x = np.exp(x)             #transfer normal distribution nodes to lognormal distribution nodes
    
    return x, prob
            
    

In [5]:
# Program 5.4 Optimal portfolio choice with risky assets
# Import package
import numpy as np
from scipy.optimize import minimize, differential_evolution

# Parameters
μwR = np.array([1.0,1.22]) #mean of labor and capital income distribution
Rf = 1.0                   #return on risk-free assets
β  = 1.0                   #discount factor
γ  = 0.5                   #risk aversion
egam = 1 - 1/γ             #denominator of utility function, power to consumption
nwR = np.array([5,5])      #number of required nodes of distribution
σwR = np.array([0.5,0.5])  #variance of the distribution
rho = -0.5                 #correlation between w and R distribution

# Endogenous variables
a2       = .0                                #asset holdinds at period 2
a3       = np.zeros(nwR[0]*nwR[1])           #possible asset holdinds at period 3
c2       = np.zeros(nwR[0]*nwR[1])           #possible consumption at period 2
Rp2      = np.zeros(nwR[0]*nwR[1])           #possible return on portfolio at period 2
Rp3      = np.zeros((nwR[0]*nwR[1],nwR[1]))  #possible return on portfolio at period 3
ω1       = .0                                #proportion to invest in risky assets in period 1
ω2       = np.zeros(nwR[0]*nwR[1])           #choices of proportion to invest in risky assets in period 2
c3       = np.zeros((nwR[0]*nwR[1],nwR[1]))  #possible consumption at period 3


# Discretize wage and R2 in period 3 and R3 in period 3
wR, weight_wR    = log_normal_discrete_2(n=nwR, mu = μwR, sigma = σwR, rho = rho) #discretized nodes' value and weight for wR
w_dis            = wR[:,0] #discretized nodes' value for w for period 2
R2_dis           = wR[:,1] #discretized nodes' value for R for period 2
R3_dis, weight_R = log_normal_discrete(nwR[1],μwR[1],σwR[1]) #discretized nodes' value for R for period 3



# Define utility function
def utility(x):
    """
    This is a 3 period utility function. 
    1st period: μwR have been realized, 
                a2 choice variable as how much to save to period 2, 
                ω1 choice variable as how much to invest in risky assets in period 1,
                c1 consumption in period 1 is pinned down by choosing a2 and ω1,
                no discount
    2nd period: discounted utility with unrealized c2, w, Rp2, so all these are arrays representing possible values of each
    3rd period: discounted for 2 periods utility with unrealized c3, a3, Rp3
    Input: asset holdings a2=x[0], ω1=x[1], a3=x[2:nwR[0]*nwR[1]+2], and ω2=x[-nwR[0]*nwR[1]:]
    Output: negative expected value of the 3 period utility function in period 1
    """
    global ω1, a2, ω2, a3, c1, c2, c3 #save ω1, a2, ω2, a3, c1, c2, c3 to global
    a2     = x[0]                     #period 2 asset holding
    ω1     = x[1]                     #period 1 investment choice
    a3     = x[2:nwR[0]*nwR[1]+2]     #period 3 asset holding
    ω2     = x[-nwR[0]*nwR[1]:]       #period 2 investment choice
    c1     = max(μwR[0] - a2,1e-10)   #period 1 consumption
    Rp2    = Rf + ω1*(R2_dis-Rf)      #period 2 return on portfolio
    c2     = np.maximum(w_dis + Rp2*a2 - a3, np.zeros(len(c2))+1e-10)  #period 2 consumption
    expect = .0                       #initial expected utility
    for i in range(len(R2_dis)):      #loop for period 2 possible capital income distribution
        for j in range(len(R3_dis)):  #loop for period 3 possible capital income distribution
            Rp3[i,j]= Rf + ω2[i]*(R3_dis[j]-Rf)  #period 3 return on portfolio
            c3[i,j] = Rp3[i,j]*a3[i]  #period 3 consumption
            c3[i,j] = max(c3[i,j], 1e-10) #period 3 verified consumption
            expect  = expect + weight_wR[i]*weight_R[j]*(c2[i]**egam+β*c3[i,j]**egam)/egam #calculating expected utility
    value = -(c1**egam/egam+β*expect) #negative value of the expected utility
    return value

# Set bounds
low                   = np.zeros(2*(1+nwR[0]*nwR[1]))       #initialize lower bound for a2, ω1, a3, and ω2
up                    = np.ones(len(low))                   #upper bound has the same length as lower bound
up[0]                 = μwR[0]                              #upper bound for a2
up[1]                 = 1.0                                 #upper bound for ω1
for i in range(2,nwR[0]*nwR[1]+2):                          #upper bounds for a3
    up[i] = w_dis[i-2]+R2_dis[i-2]*μwR[0]
up[-nwR[0]*nwR[1]:]   = 1.0                                 #upper bounds for ω2
bnds = [(low[i],up[i]) for i in range(2*(1+nwR[0]*nwR[1]))] #summarize bounds

# Initial guess
x0 = up/2.0

#res = minimize(utility,x0,method='Powell',bounds=bnds)
res = differential_evolution(utility,bounds=bnds)   #Finds the global minimum of a multivariate function.


# Calculate the expectation
# Expectation of 2D array
def E(x):
    E = 0.0
    if np.size(x) == nwR[0]*nwR[1]:
        for i in range(len(x)):
            E = E + weight_wR[i]*x[i]
    else:
        for i in range(nwR[0]*nwR[1]):
            for j in range(nwR[1]):
                E = E + weight_wR[i]*weight_R[j]*x[i,j]
    return E
# Calculate the standard deviation
# Standard deviation of 2D array
def Std(x):
    E2  = 0.0
    std = 0.0
    if np.size(x) == nwR[0]*nwR[1]:
        for i in range(len(x)):
            E2 = E2 + weight_wR[i]*x[i]**2
        std = E2 - E(x)**2
    else:
        for i in range(nwR[0]*nwR[1]):
            for j in range(nwR[1]):
                E2 = E2 + weight_wR[i]*weight_R[j]*x[i,j]**2
        std = E2 - E(x)**2
    return std


print('-------------------')
print('First period:')
print('c1      = ',"%.2f"%c1     , 'a2      = ',"%.2f"%a2     , 'ω1      = ',"%.2f"%ω1)
print('-------------------')
print('Second period:')
print('E(c2)   = ',"%.2f"%E(c2)  , 'E(a3)   = ',"%.2f"%E(a3)  , 'E[ω2]   = ',"%.2f"%E(ω2))
print('Std(c2) = ',"%.2f"%Std(c2), 'Std(a3) = ',"%.2f"%Std(a3), 'Std[ω2] = ',"%.2f"%Std(ω2))
print('-------------------')
print('Third period:')
print('E(c3)   = ',"%.2f"%E(c3))
print('Std(c2) = ',"%.2f"%Std(c2))
print('-------------------')

-------------------
First period:
c1      =  0.59 a2      =  0.41 ω1      =  1.00
-------------------
Second period:
E(c2)   =  0.76 E(a3)   =  0.75 E[ω2]   =  0.33
Std(c2) =  0.10 Std(a3) =  0.09 Std[ω2] =  0.00
-------------------
Third period:
E(c3)   =  0.80
Std(c2) =  0.10
-------------------


In [6]:
### Program 5.5 Lifespan uncertainty and annuity choice
# Import package
import numpy as np
from scipy.optimize import minimize, differential_evolution

# Parameters
R    = 1.0                  #return on asset holdings
β    = 1.0                  #discount factor
μw   = 1.0                  #mean of labor income distribution
pen  = 1.0                  #pension in the last period
γ    = 0.5                  #risk aversion
ψ    = np.array([0.8,0.5])  #survival probabilities
nw   = 5                    #number of nodes needed for discretizing labour income distribution
σw    = 0.3                 #variance of discretizing labour income distribution
Pa   = np.array([ψ[0]/R + ψ[0]*ψ[1]/(R**2),ψ[1]/R])  #price of annuity in period 1 and period 2
egam = 1 - 1/γ              #denominator of utility function, power to consumption
a_low = 0.0                 #lower bound value for a2 in period 1


# Discretize income
w_dis, weight = log_normal_discrete(nw, mu = μw, sigma = σw) #discretized nodes' value and weight for w


# Objective function
def utility(x):
    """
    This is a 3 period utility function. 
    1st period: μw has been realized, 
                a2 choice variable as how much to save to period 2, 
                ω1 choice variable as how much to invest in annuity in period 1,
                c1 consumption in period 1 is pinned down by choosing a2 and ω1,
                no discount
    2nd period: discounted utility with unrealized c2, w, a3, so all these are arrays representing possible values of each
    3rd period: discounted for 2 periods utility with unrealized c3, a3
    Input: asset holdings a2=x[0], ω1=x[1], a3=x[2:nw+2], and ω2=x[-nw:]
    Output: negative expected value of the 3 period utility function in period 1
    """
    global c1, a2, c2, a3, c3, ω1, ω2
    a2     =  x[0]              #period 2 asset holding
    ω1     =  x[1]              #period 1 investment choice
    a3     =  x[2:nw+2]         #period 3 asset holding
    ω2     =  x[-nw:]           #period 3 investment choice
    c1     =  max(μw-a2,1e-20)  #period 1 consumption
    c2     =  np.maximum(w_dis + R*(1-ω1)*a2 + ω1*a2/Pa[0] - a3       , np.zeros(nw)+1e-20)  #period 2 consumption
    c3     =  np.maximum(pen + R*(1-ω2)*a3 + ω2*a3/Pa[1] + ω1*a2/Pa[0], np.zeros(nw)+1e-20)  #period 3 consumption
    expect =  0.0               #initial expected utility
    for i in range(nw):
        expect = expect + weight[i]*(c2[i]**egam + ψ[1]*β*c3[i]**egam)/egam #utility optimization problem
    value = -(c1**egam/egam + ψ[0]*β*expect)  #negative value of utility function
    return value

# Set bounds
low         = np.zeros(2*nw+2) #lower bounds for a2, ω1, a3, ω2
low[0]      = a_low            #Set lower bound for 'a'. In the last case, we set lower bound of 'a' to be negative infinity.
low[2:nw+2] = low[0]           #set all lower bounds to a_low

up    = np.ones(len(low))      #upper bound has the same length as lower bound
up[0] = μw                     #upper bound of a2
for i in range(2,nw+2):        #upper bound of a3
    up[i] = w_dis[i-2] + R*μw
    
bnds = [(low[i],up[i]) for i in range(2*nw+2)]  #summarize bounds

# Initial Guess
x0 = up/2.0


# Minimise routine
#res = minimize(utility, x0, bounds = bnds, tol=1e-14)
res = differential_evolution(utility, bounds=bnds, tol = 1e-14)  #Finds the global minimum of a multivariate function.


# Other endogenous variables
a2a = ω1*a2       #period 2 Assets spend on annuity
a2b = (1-ω1)*a2   #period 2 Assets spend on bonds
a3a = ω2*a3       #period 3 Assets spend on annuity 
a3b = (1-ω2)*a3   #period 3 Assets spend on bonds

# Calculate the expectation
def E(x):
    E  = 0.0
    for i in range(len(x)):
        E = E + weight[i]*x[i]
    return E
# Calculate the standard deviation
def Std(x):
    E2  = 0.0
    std = 0.0
    for i in range(len(x)):
        E2 = E2 + weight[i]*x[i]**2
    std = E2 - E(x)**2
    return std

print('-------------------')
print('First period:')
print('c1      = ',"%.2f"%c1     , 'a2      = ',"%.2f"%a2b     , 'a2a      = ',"%.2f"%a2a     )
print('-------------------')
print('Second period:')
print('E(c2)   = ',"%.2f"%E(c2)  , 'E(a3)   = ',"%.2f"%E(a3b)  , 'E(a3a)   = ',"%.2f"%E(a3a)  )
print('Std(c2) = ',"%.2f"%Std(c2), 'Std(a3) = ',"%.2f"%Std(a3b), 'Std[a3a] = ',"%.2f"%Std(a3a))
print('-------------------')
print('Third period:')
print('E(c3)   = ',"%.2f"%E(c3))
print('Std(c2) = ',"%.2f"%Std(c2))
print('-------------------')

-------------------
First period:
c1      =  0.88 a2      =  0.00 a2a      =  0.12
-------------------
Second period:
E(c2)   =  1.03 E(a3)   =  0.00 E(a3a)   =  0.07
Std(c2) =  0.18 Std(a3) =  0.00 Std[a3a] =  0.02
-------------------
Third period:
E(c3)   =  1.23
Std(c2) =  0.18
-------------------
